# Delay Analysis: Temporal and Spatial
Delay data of DB was collected over several months with DB API. Planned time and actual time of departure were collected for each train. Analysis of delays by time of day, day of week, and station location.

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import warnings

# Suppress warnings for cleaner output (e.g., FutureWarning, DeprecationWarning)
warnings.filterwarnings('ignore')

sns.set_theme(style="whitegrid")

In [2]:
# Connect to the SQLite database
# The database contains timetable and station information.
db_path = '../data/amse.sqlite'
conn = sqlite3.connect(db_path)

def get_table_names(conn):
    """Retrieve all table names from the connected SQLite database."""
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [x[0] for x in cursor.fetchall()]

# Identify all timetable tables (sharded by name)
tables = get_table_names(conn)
timetable_tables = [t for t in tables if t.startswith('timetables')]

In [3]:
# --- Global Data Loading ---
print("Loading and Aggregating All Train Data...")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE 'timetables%'")
tables = [row[0] for row in cursor.fetchall()]

all_data = []
for table in tables:
    try:
        # Query necessary columns for all analyses: pt, ct, eva
        df = pd.read_sql_query(f'SELECT "pt", "ct", "eva" FROM "{table}"', conn)
        
        # --- Data Cleaning & Imputation ---
        # Drop missing planned times
        df = df.dropna(subset=['pt'])
        
        # Handle string 'nan' if present and fill missing ct with pt (assuming on time)
        df['ct'] = df['ct'].fillna(df['pt'])
        # Check for string 'nan' just in case data has it
        mask_nan = (df['ct'].astype(str) == 'nan')
        df.loc[mask_nan, 'ct'] = df.loc[mask_nan, 'pt']
        
        # Convert to datetime
        df['pt_dt'] = pd.to_datetime(df['pt'], format='%d.%m.%Y, %H:%M', errors='coerce')
        df['ct_dt'] = pd.to_datetime(df['ct'], format='%d.%m.%Y, %H:%M', errors='coerce')
        
        # Filter out rows where parsing failed
        df = df.dropna(subset=['pt_dt'])
        # If ct failed to parse but pt didn't, assume on time (fill again with parsed pt)
        df['ct_dt'] = df['ct_dt'].fillna(df['pt_dt'])
        
        # Calculate delay
        df['delay_min'] = (df['ct_dt'] - df['pt_dt']).dt.total_seconds() / 60.0
        
        # Extract features
        df['hour'] = df['pt_dt'].dt.hour
        df['date'] = df['pt_dt'].dt.date
        df['day_of_week'] = df['pt_dt'].dt.dayofweek
        df['date_str'] = df['date'].astype(str)
        
        # Store processed chunk
        all_data.append(df[['eva', 'delay_min', 'hour', 'day_of_week', 'date', 'date_str', 'pt_dt']])
    except Exception as e:
        pass

if all_data:
    train_delays = pd.concat(all_data)
    print(f"Loaded {len(train_delays)} records.")
else:
    train_delays = pd.DataFrame()
    print("No data loaded.")


Loading and Aggregating All Train Data...
Loaded 464286 records.


In [4]:
# --- Time of Day & Time Series Analysis ---
if not train_delays.empty:
    # Daily Stats for Subplot 1
    daily_stats = train_delays.groupby('date')['delay_min'].agg(['mean']).reset_index()

    # Weekly Stats for Subplot 2
    days = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
    train_delays['day_name'] = train_delays['day_of_week'].map(days)
    weekly_stats = train_delays.groupby('day_name')['delay_min'].agg(['mean', 'std', 'count']).reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']).reset_index()
    weekly_stats['sem'] = weekly_stats['std'] / np.sqrt(weekly_stats['count'])
    
    # Hourly Stats for Subplot 3 and 4
    hourly_stats = train_delays.groupby('hour')['delay_min'].agg(['mean', 'std', 'count']).reset_index()
    hourly_stats['sem'] = hourly_stats['std'] / np.sqrt(hourly_stats['count'])
    

    # --- Visualization: Combined Subplots ---
    fig = make_subplots(rows=1, cols=4, 
                        subplot_titles=("By Date", "By Day of Week", "By Hour of Day", "By Hour of Day (sum)"),
                        horizontal_spacing=0.08)

    # Plot A
    fig.add_trace(go.Scatter(
        x=daily_stats['date'], y=daily_stats['mean'],
        mode='lines', name='Daily'
    ), row=1, col=1)

    # Plot B
    fig.add_trace(go.Bar(
        x=weekly_stats['day_name'], y=weekly_stats['mean'],
        error_y=dict(type='data', array=weekly_stats['sem'], visible=True),
        name='Weekly'
    ), row=1, col=2)

    # Plot C
    fig.add_trace(go.Scatter(
        x=hourly_stats['hour'], y=hourly_stats['mean'],
        error_y=dict(type='data', array=hourly_stats['sem'], visible=True),
        mode='lines+markers', name='Hourly'
    ), row=1, col=3)

    # Plot D
    fig.add_trace(go.Bar(
        x=hourly_stats['hour'], y=hourly_stats['count'],
        error_y=dict(type='data', array=hourly_stats['sem'], visible=True),
         name='Hourly'
    ), row=1, col=4)

    
    fig.update_layout(
        title_text="Avg Delay per Train Analysis", showlegend=False, height=400,
        template="plotly_white", margin=dict(l=40, r=40, t=60, b=40)
    )
    fig.update_yaxes(title_text="Avg Delay (min)", row=1, col=1)
    fig.show()

    # --- Visualization: Heatmap ---
    heatmap_data = train_delays.groupby(['day_name', 'hour'])['delay_min'].mean().reset_index()
    pivot_table = heatmap_data.pivot(index='day_name', columns='hour', values='delay_min')
    pivot_table = pivot_table.reindex(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    
    fig_heat = px.imshow(pivot_table,
                         labels=dict(x="Hour of Day", y="Day of Week", color="Avg Delay (min)"),
                         x=pivot_table.columns, y=pivot_table.index,
                         color_continuous_scale='RdBu_r', 
                         title="Interaction of Time and Day on Average Delay")
    fig_heat.show()


In [6]:
# --- Spatial Analysis ---
print("Running Spatial Analysis...")
try:
    # Load Station Coordinates
    geo_df = pd.read_sql_query("SELECT * FROM geo_data_train_stations", conn)
    print("Loaded geo_data_train_stations.")
    
    # Identify coordinate columns
    lat_col = next((c for c in geo_df.columns if 'lat' in c.lower()), None)
    lon_col = next((c for c in geo_df.columns if 'lon' in c.lower() or 'lng' in c.lower()), None)
    geo_eva_col = next((c for c in geo_df.columns if 'eva' in c.lower()), None)
    
    if lat_col and lon_col and geo_eva_col:
        if not train_delays.empty:
            # Use aggregated stats from train_delays
            # --- 1. Static Profile ---
            station_stats = train_delays.groupby('eva')['delay_min'].mean().reset_index()
            
            # Join with Geo Data
            station_stats['eva'] = station_stats['eva'].astype(str)
            geo_df[geo_eva_col] = geo_df[geo_eva_col].astype(str)
            merged = station_stats.merge(geo_df, left_on='eva', right_on=geo_eva_col, how='inner')
            
            print("Plotting Static Map...")
            fig3 = px.scatter_mapbox(merged, lat=lat_col, lon=lon_col, hover_name=geo_eva_col,
                                     color='delay_min', size='delay_min',
                                     color_continuous_scale='RdBu_r', 
                                     size_max=15, zoom=4,
                                     mapbox_style="carto-positron", 
                                     title="Average Delay per Station",
                                     height=500, width=400,
                                     hover_data={ 'delay_min': ':.2f'})
            fig3.show()
            
            # --- 2. Temporal Animation (Time Slider) ---
            print("Generating Map with Time Slider...")
            # Aggregate daily per station
            daily_spatial = train_delays.groupby(['eva', 'date_str'])['delay_min'].mean().reset_index()
            daily_spatial = daily_spatial.rename(columns={'date_str': 'date'}) # expect 'date' col for animation
            
            daily_spatial['eva'] = daily_spatial['eva'].astype(str)
            merged_daily = daily_spatial.merge(geo_df, left_on='eva', right_on=geo_eva_col, how='inner')
            merged_daily = merged_daily.sort_values('date')
            merged_daily['abs_delay'] = merged_daily['delay_min'].abs()
            
            c_min = merged_daily['delay_min'].min()
            c_max = merged_daily['delay_min'].max()
            
            fig4 = px.scatter_mapbox(merged_daily, lat=lat_col, lon=lon_col, hover_name=geo_eva_col,
                                     color='delay_min', size='abs_delay',
                                     animation_frame='date',
                                     color_continuous_scale='RdBu_r', 
                                     range_color=[c_min, c_max],
                                     size_max=15, zoom=4,
                                     mapbox_style="carto-positron",
                                     title="Average Delay per Station over Time",
                                     height=500, width=400,
                                     hover_data={'abs_delay': False, 'delay_min': ':.2f'})
            fig4.show()
        else:
            print("No train delay data available for spatial analysis.")
    else:
        print(f"Could not identify necessary columns in geo_data_train_stations.")
except Exception as e:
    print(f"Spatial analysis failed: {e}")


Running Spatial Analysis...
Loaded geo_data_train_stations.
Plotting Static Map...


Generating Map with Time Slider...
